In [1]:
# set Environment Variable in jupyter notebook using magic command %env
# %env to see environment variable available
# %env MY_VAR=MY_VALUE to set environment variable

import os
import time
import datetime as dt
import pandas as pd
from binance.client import Client
from binance import ThreadedWebsocketManager, BinanceSocketManager
from binance.enums import *
from twisted.internet import reactor

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [ ]:
# https://testnet.binance.vision/  >> for test API

# api_key = os.environ.get('test_binance_api')
# api_secret = os.environ.get('test_binance_secret')

api_key = 'UeRMEYWx3LWuWFFt47Y5jd94j9w2qCQtWqSJfSySz4PLjbH7gkzwPZX7Nh9mxEnl'
api_secret = '9oCJjPb9Hxb9czkTnjXRXRTWGkUh2RbVbaLyFW3uz3lAv3uZBSvs9MhshttEDSLW'

In [ ]:
client = Client(api_key, api_secret, testnet=True)
btc_price = {}

In [ ]:
def handle_socket_message(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        btc_price['close'] = msg['k']['c']
        btc_price['high'] = msg['k']['h']
        btc_price['low'] = msg['k']['l']
        btc_price['volume'] = msg['k']['v']
    else:
        btc_price['error'] = True
        
    print(btc_price['close'])

In [ ]:
# def handle_socket_message(msg):
#     ''' define how to process incoming WebSocket messages '''
#     print(f"message type: {msg['e']}")
#     print(msg)

In [ ]:
symbol = 'BTCUSDT'
twm = ThreadedWebsocketManager(api_key=api_key, api_secret=api_secret, testnet=True)
twm.start()
connection = twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

In [ ]:
twm.stop_socket(connection)
# reactor.stop()

In [ ]:
twm.is_alive()

### Binance SMA Strategy Testing

In [13]:
#write algo class for simple SMA including data resampling

# connection for streaming and place order
# symbol, units, buy_order, sell_order,
# self.data = pd.DataFrame()
# self.ticks = 0
# self.position = 0


'''for threaded websocket manager the connection requires api_key and api_secret instead of client arg*
declare self.con init as a threadedWebsocketMaager directly, do not have to declare client first
ThreadedWebsocketManager is an asynchronous. Once connection start can directly callback function which 
is SMA_trade receiving msg and directly append to dataframe then process the strategy before firing the order
'''    
api_key = 'UeRMEYWx3LWuWFFt47Y5jd94j9w2qCQtWqSJfSySz4PLjbH7gkzwPZX7Nh9mxEnl'
api_secret = '9oCJjPb9Hxb9czkTnjXRXRTWGkUh2RbVbaLyFW3uz3lAv3uZBSvs9MhshttEDSLW'

class SMATrading:
    
    def __init__(self, symbol, units):
#         initializing the trading class
        self.con = ThreadedWebsocketManager(api_key=api_key, api_secret=api_secret, testnet=True)
        self.client = Client(api_key, api_secret, testnet = True)
        self.symbol = str(symbol)
        self.units = int(units)
#         self.buy_order = self.con.order_market_buy(self.symbol, self.units)
#         self.sell_order = self.con.order_market_sell(self.symbol, self.units)
        self.data = pd.DataFrame()
        self.ticks = 0
        self.position = 0
        self.timestamp = dt.datetime.now()
    
    def streaming(self):
        self.con.start()
        connection = self.con.start_kline_socket(callback=self.SMA_Trade, symbol = self.symbol)
        
#     def handle_mssg(self, msg):
#         ''' define how to process incoming WebSocket messages '''
# #         need to define the data structure for msg
#         if msg['e'] != 'error':
#             btc_price['close'] = msg['k']['c']
#             btc_price['high'] = msg['k']['h']
#             btc_price['low'] = msg['k']['l']
#             btc_price['volume'] = msg['k']['v']
#         else:
#             btc_price['error'] = True
    
    def SMA_Trade(self, msg):
        if msg['e'] != 'error':
            kline = msg['k']
            Close = kline['c']
            self.ticks += 1
            self.data = self.data.append(pd.DataFrame({'Close': kline['c']}, index = [self.timestamp]), sort = True)
            
        self.data['SMA1'] = self.data['Close'].rolling(5).mean().round(2)
        self.data['SMA2'] = self.data['Close'].rolling(10).mean().round(2)
        
        SMA1 = self.data['SMA1'][-1]
        SMA2 = self.data['SMA2'][-1]
        print(f'{self.ticks} | {self.timestamp} | {Close} | SMA1 {SMA1} | SMA2 {SMA2}')
        
        '''
        still missing data resampling in this code, need to adjust after connection testing complete
                
        '''
        
        if (self.data['SMA1'].iloc[-1] > self.data['SMA2'].iloc[-1]) and (self.position == 0):
            self.data['Signal'] = 1
            print('Create | BUY ORDER')
            Symbol = self.symbol
            self.order = self.client.create_test_order(
            symbol = 'BTCUSDT',
            side = SIDE_BUY,
            type = ORDER_TYPE_LIMIT,
            timeInForce = TIME_IN_FORCE_GTC,
            quantity = self.units,
            price = self.data['Close'][-1]
            )
            self.position = 1
        
        elif (self.data['SMA1'].iloc[-1] < self.data['SMA2'].iloc[-1]) and (self.position == 1):
            self.data['Signal'] = -1
            print('Create | SELL ORDER')
            Symbol = self.symbol
            self.order = self.client.create_test_order(
            symbol = 'BTCUSDT',
            side = SIDE_SELL,
            type = ORDER_TYPE_LIMIT,
            timeInForce = TIME_IN_FORCE_GTC,
            quantity = self.units,
            price = self.data['Close'][-1]
            )
            self.position = 0
    
    def stop_strategy(self):
        self.con.stop()
    
# if __name__ == '__main__':
#     sma = SMATrading('BTC', 2000)
#     time.sleep(7)
#     sma.streaming()
#     while sma.con.is_alive():
#         pass
        

# adjust to include Trading ID and filled price
# adjust to include method to easily request historical trade
# adjust create_test_order to be able to take self.symbol, self.units
# adjust class SMATrading to be able to input SMA1, SMA2 args plus condition to check SMA1 to be less than SMA2
# prevent order from firing after the first SMA has been calculated

# backtesting class to find the best optimal SMA combinations.
# check timestamp since it remains the same throughout the connection

SyntaxError: invalid syntax (<ipython-input-13-01d2d5ff9574>, line 59)

In [3]:
sma = SMATrading('BTCUSDT', 200)

In [4]:
sma.streaming()

1 | 2021-06-22 13:18:45.441048 | 32670.01000000 | SMA1 nan | SMA2 nan
2 | 2021-06-22 13:18:45.441048 | 32671.83000000 | SMA1 nan | SMA2 nan
3 | 2021-06-22 13:18:45.441048 | 32680.04000000 | SMA1 nan | SMA2 nan
4 | 2021-06-22 13:18:45.441048 | 32681.00000000 | SMA1 nan | SMA2 nan
5 | 2021-06-22 13:18:45.441048 | 32600.00000000 | SMA1 32660.58 | SMA2 nan
6 | 2021-06-22 13:18:45.441048 | 32686.54000000 | SMA1 32663.88 | SMA2 nan
7 | 2021-06-22 13:18:45.441048 | 32693.14000000 | SMA1 32668.14 | SMA2 nan
8 | 2021-06-22 13:18:45.441048 | 32679.68000000 | SMA1 32668.07 | SMA2 nan
9 | 2021-06-22 13:18:45.441048 | 32684.34000000 | SMA1 32668.74 | SMA2 nan
10 | 2021-06-22 13:18:45.441048 | 32688.91000000 | SMA1 32686.52 | SMA2 32673.55
Create | BUY ORDER
11 | 2021-06-22 13:18:45.441048 | 32748.00000000 | SMA1 32698.81 | SMA2 32681.35
12 | 2021-06-22 13:18:45.441048 | 32694.12000000 | SMA1 32699.01 | SMA2 32683.58
13 | 2021-06-22 13:18:45.441048 | 32693.74000000 | SMA1 32701.82 | SMA2 32684.95
14

In [5]:
sma.stop_strategy()

34 | 2021-06-22 13:18:45.441048 | 32674.27000000 | SMA1 32667.27 | SMA2 32682.46


In [12]:
sma.client.get_historical_trades(symbol='BTCUSDT')

[{'id': 848114,
  'price': '32797.28000000',
  'qty': '0.01524400',
  'quoteQty': '499.96173632',
  'time': 1624343084110,
  'isBuyerMaker': False,
  'isBestMatch': True},
 {'id': 848115,
  'price': '32799.99000000',
  'qty': '0.01524400',
  'quoteQty': '500.00304756',
  'time': 1624343084348,
  'isBuyerMaker': False,
  'isBestMatch': True},
 {'id': 848116,
  'price': '32900.00000000',
  'qty': '0.00060900',
  'quoteQty': '20.03610000',
  'time': 1624343084348,
  'isBuyerMaker': False,
  'isBestMatch': True},
 {'id': 848117,
  'price': '32912.00000000',
  'qty': '0.00608300',
  'quoteQty': '200.20369600',
  'time': 1624343084348,
  'isBuyerMaker': False,
  'isBestMatch': True},
 {'id': 848118,
  'price': '32798.55000000',
  'qty': '0.00060000',
  'quoteQty': '19.67913000',
  'time': 1624343085952,
  'isBuyerMaker': False,
  'isBestMatch': True},
 {'id': 848119,
  'price': '32790.01000000',
  'qty': '0.00548400',
  'quoteQty': '179.82041484',
  'time': 1624343087252,
  'isBuyerMaker': F

In [6]:
help(BinanceSocketManager)

Help on class BinanceSocketManager in module binance.streams:

class BinanceSocketManager(builtins.object)
 |  BinanceSocketManager(client: binance.client.AsyncClient, loop=None, user_timeout=300)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, client: binance.client.AsyncClient, loop=None, user_timeout=300)
 |      Initialise the BinanceSocketManager
 |      
 |      :param client: Binance API client
 |      :type client: binance.AsyncClient
 |  
 |  aggtrade_futures_socket(self, symbol: str, futures_type: binance.enums.FuturesType = <FuturesType.USD_M: 1>)
 |      Start a websocket for aggregate symbol trade data for the futures stream
 |      
 |      :param symbol: required
 |      :param futures_type: use USD-M or COIN-M futures default USD-M
 |      
 |      :returns: connection key string if successful, False otherwise
 |      
 |      Message Format
 |      
 |      .. code-block:: python
 |      
 |          {
 |              "e": "aggTrade",  // Event type
 |          

In [ ]:
help(ThreadedWebsocketManager)

In [9]:
help(Client)

Help on class Client in module binance.client:

class Client(BaseClient)
 |  Client(api_key: Union[str, NoneType] = None, api_secret: Union[str, NoneType] = None, requests_params: Dict[str, str] = None, tld: str = 'com', testnet: bool = False)
 |  
 |  Method resolution order:
 |      Client
 |      BaseClient
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, api_key: Union[str, NoneType] = None, api_secret: Union[str, NoneType] = None, requests_params: Dict[str, str] = None, tld: str = 'com', testnet: bool = False)
 |      Binance API Client constructor
 |      
 |      :param api_key: Api Key
 |      :type api_key: str.
 |      :param api_secret: Api Secret
 |      :type api_secret: str.
 |      :param requests_params: optional - Dictionary of requests params to use for all calls
 |      :type requests_params: dict.
 |      :param testnet: Use testnet environment - only available for vanilla options at the moment
 |      :type testnet: bool
 |  
 |  aggre